In [34]:
import re
from random import uniform
from collections import defaultdict
import json
import io
from transiltor import ScopusExtractor

#r_alphabet = re.compile(u'[а-яА-Я0-9-]+|[.,:;?!]+')
r_alphabet = re.compile(u'[a-zA-Z0-9-]+|[.,:;?!$]+')

def gen_lines(corpus):
    #data = open(corpus, encoding='utf-8')
    #for line in data:
    return corpus.lower()
        #line.lower()
def gen_tokens(lines):
    #for line in lines:
    for token in r_alphabet.findall(lines):
        yield token

def gen_trigrams(tokens):
    t0, t1 = '$', '$'
    for t2 in tokens:
        yield t0, t1, t2
        if t2 in '.!?':
            yield t1, t2, '$'
            yield t2, '$','$'
            t0, t1 = '$', '$'
        else:
            t0, t1 = t1, t2

def train_trigram(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)

    bi, tri = defaultdict(lambda: 0.0), defaultdict(lambda: 0.0)

    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1

    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]
    return model

def generate_sentence(model):
    phrase = ''
    t0, t1 = '$', '$'
    while 1:
        t0, t1 = t1, unirand(model[t0, t1])
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            phrase += ' ' + t1
    return phrase.capitalize()

def unirand(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return token
        
def abstract_summator(path_to_json_file, input_name):
    papers_json=[]
    json_file = io.open('texts/users_articles_data_2.json',encoding='utf-8')
    for line in json_file:
        papers_json.append(json.loads(json_file.readline()))
    abstract_sum=[]
    extractor = ScopusExtractor('users_articles_data_2.json', d1=0.5, f1=0.5)
    english_author_name = extractor.try_get_scopus_ids(input_name)
    name = list(english_author_name[0])[0]
    print(name)
    for i in range(len(papers_json)):
        if ('authorlist' in list(papers_json[i].keys())):
            for names in papers_json[i]['authorlist']:
                if (name in names.lower()):
                    abstract_sum.append(papers_json[i]['abstract'])
    abstract_sum=' '.join(abstract_sum)       
    return abstract_sum
        
def korchevatel(path_to_json_file, input_name, number_of_sentences=2):
    generated_text=[]
    abstract_sum = abstract_summator(path_to_json_file, input_name)
    model = train_trigram(abstract_sum)
    for i in range(number_of_sentences):
        generated_text.append(generate_sentence(model))
    generated_text=' '.join(generated_text) 
    return generated_text


In [36]:
path_to_json_file = 'texts/users_articles_data_2.json'
input_name = 'Гулева Вал'
number_of_sentences = 3
korchevatel(path_to_json_file, input_name, 2)


guleva, v.


'Current paper propose agent-based model of a timeline into branches. Any modifications in simulation parameters result in division of a timeline into branches.'